In [ ]:
from DB.db import dbcon
import requests
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
import json

In [2]:
# 상품 목록 저장하는 로직 

res = requests.get("https://store.ohou.se/_next/data/Ion4JL5aF_XBJaflIPJ3S/ko-KR/ranks.json?type=best&category_id=18000000&__state__=%7B%22key%22%3A%22nny2ak49%22%7D",
                   headers={
                       'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
                        'Sec-Fetch-Dest': 'empty',
                        'Sec-Fetch-Mode': 'cors',
                        'Sec-Fetch-Site': 'same-origin'})
res.status_code
data = res.json()
# 상품 DB 저장할 컬럼 브랜드명, 제품명
goods_lists = data['pageProps']['dehydratedState']['queries'][1]['state']['data']['products'] # 브랜드명, 제품명

result = []
for goods_list in goods_lists:
    result.append({
        '상품ID' : goods_list['id'],
        '브랜드명' : goods_list['brandName'],
        '제품명' : goods_list['name'],
    })
result # 총 20개

[{'상품ID': 767440,
  '브랜드명': '헬로우슬립',
  '제품명': '[7일간,최대20%쿠폰]NEW컬러! 카스테라 워싱 옥수수솜 간절기/사계절 차렵이불세트'},
 {'상품ID': 212240,
  '브랜드명': '아엠홈',
  '제품명': '[5%쿠폰][맞춤]비침없는 도톰 레이스/쉬폰커튼(나비주름/핀형/봉집)'},
 {'상품ID': 264565, '브랜드명': '데코지오', '제품명': '맞춤 브라우니 린넨 암막커튼 핀형/아일렛형/형상기억 33색상'},
 {'상품ID': 169814,
  '브랜드명': '스타일링홈',
  '제품명': '[1+1]100% 완벽 자외선 빛차단 냉방유지 창문/거실/안방 아일렛/핀형 암막커튼'},
 {'상품ID': 676405,
  '브랜드명': '베이직톤',
  '제품명': '[단하루!20%쿠폰]호텔침구 클린코튼 고밀도60수 사계절/간절기 옥수수솜 차렵이불세트'},
 {'상품ID': 2636441,
  '브랜드명': '쁘리엘르',
  '제품명': '[단하루 역대급 혜택] NEW 재구매율 높은 퐁신퐁신 유어메이트 항균 차렵이불 풀세트'},
 {'상품ID': 3316539, '브랜드명': '드파인', '제품명': '하이랜드 루프 사이잘룩 러그 7size'},
 {'상품ID': 33583, '브랜드명': '창안애', '제품명': '25mm 알루미늄 블라인드 거실 창문 35colors'},
 {'상품ID': 338170,
  '브랜드명': '까르데코',
  '제품명': '메리미 화이트 이중 레이스 암막커튼 2장세트 핀형/아일렛형4size'},
 {'상품ID': 2370041,
  '브랜드명': '마틸라',
  '제품명': '[기간한정,쿠폰15%+무배]NEW빌리호텔 먼지없는 차렵이불세트 23color'},
 {'상품ID': 1215312,
  '브랜드명': '체고루루',
  '제품명': '[오늘의딜]디아망 방수러그 먼지없는 거실 대형 워셔블 카페트 고양이 러그 18color'},
 {'상품ID': 1418521,
  '브랜드명

In [8]:
len(result)
result[0]['상품ID']

767440

In [ ]:
# DB에 상품정보 저장하기 
conn = dbcon()

with conn.cursor() as cur:
    sql = "INSERT INTO tb_products (productID, brand_name, product_name) VALUES (%s, %s, %s)"
    
    for data in result:
        values = (
            data['상품ID'],
            data['브랜드명'],
            data['제품명'],
        )
        cur.execute(sql, values)
        
conn.commit()

In [ ]:
pro_result = []

# 여러 페이지 데이터 수집
for j in range(0, len(result)):
    product_id = result[j]['상품ID']
    for i in range(4,5):
        url = f"https://ohou.se/production_reviews.json?production_id={product_id}&page={i}&order=recent&photo_review_only="
        pro = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin'
        })
        pro_result.append(pro.json())

    reviews = []

    for review in pro_result:   # pro_result = [page1, page2, ...]
        review_lists = review['reviews']
        for review_list in review_lists:
            reviews.append({
                '리뷰ID': review_list['id'],
                '상품ID': review_list['production_information']['id'],
                '고객ID': review_list['writer_id'],
                '고객닉네임': review_list['writer_nickname'],
                '상품옵션': review_list['production_information']['explain'],
                '별점': review_list['review']['star_avg'],
                '작성내용': review_list['review']['comment'],
                '작성날짜': review_list['created_at'],
            })

    reviews 

[{'리뷰ID': 53298576,
  '상품ID': 767440,
  '고객ID': 8919646,
  '고객닉네임': 'L아나',
  '상품옵션': '색상: 크림아이보리 / 구성 및 사이즈: 퀸&킹겸용 이불베개세트(Q&K이불+베개커버2P)',
  '별점': 5.0,
  '작성날짜': '2025.08.19',
  '작성내용': '이불 색감도 딱 마음에 들고 무엇보다 촉감이랑 두께가 마음에 딱 들어서 이제 거의 애착 이불 됐어요ㅠㅠ여름에도 몸에 말고 자면 시원하고 겨울에는 너무 따뜻해요'},
 {'리뷰ID': 53297483,
  '상품ID': 767440,
  '고객ID': 3828563,
  '고객닉네임': 'bominuri',
  '상품옵션': '색상: 간절기용_모카브라운 / 구성 및 사이즈: 슈퍼싱글 이불베개세트(SS)',
  '별점': 5.0,
  '작성날짜': '2025.08.19',
  '작성내용': '최고예요'},
 {'리뷰ID': 53297346,
  '상품ID': 767440,
  '고객ID': 10988259,
  '고객닉네임': '워니뭐니',
  '상품옵션': '색상: [단독] 간절기용_오션블루 / 구성 및 사이즈: 슈퍼싱글 풀세트(SS)',
  '별점': 5.0,
  '작성날짜': '2025.08.19',
  '작성내용': '간절기용으로 딱~~~이네요\n부들부들 가볍구 좋아요\n색상은 파스텔톤입니다~~^^'},
 {'리뷰ID': 53297342,
  '상품ID': 767440,
  '고객ID': 10988259,
  '고객닉네임': '워니뭐니',
  '상품옵션': '색상: 간절기용_그레이 / 구성 및 사이즈: 슈퍼싱글 풀세트(SS)',
  '별점': 5.0,
  '작성날짜': '2025.08.19',
  '작성내용': '울 딸 가볍구 부들부들 좋다고 하네요\n색상은 파스텔톤이예요 좋아요'},
 {'리뷰ID': 53297113,
  '상품ID': 767440,
  '고객ID': 4942433,
  '고객닉네임': 'candy0903',
 

In [ ]:
# DB에 상품정보 저장하기 
conn = dbcon()

with conn.cursor() as cur:
    sql_product_id = "SELECT ID FROM tb_products WHERE productID = %s"
    sql = "INSERT INTO tb_reviews (goodsID, reviewID, customerID, nickname, options, grade, comment, event_date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    
    for review in reviews:
        # 상품ID 가져오기 
        product_id = review['상품ID']
        cur.execute(sql_product_id, (product_id,))
        product = cur.fetchone()
        goodsID = product[0]
        
        values = (
            goodsID,
            review['리뷰ID'],
            review['고객ID'],
            review['고객닉네임'],
            review['상품옵션'],
            float(review['별점']),
            review['작성내용'],
            review['작성날짜'],
        )
        cur.execute(sql, values)
        
conn.commit()

In [13]:
from DB.db import dbcon
import requests


In [ ]:
def product_list():
    url = "https://store.ohou.se/_next/data/Zbh8EyPGpBn2SvVTElpdg/ko-KR/ranks.json?type=best&category_id=18000000"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin'
    }

    # 요청 보내기
    res = requests.get(url, headers=headers)
    print("응답 상태 코드:", res.status_code)  # 상태 코드 확인

    # JSON 파싱 안전하게
    try:
        data = res.json()
    except ValueError:
        print("JSONDecodeError 발생! 서버 응답 내용 일부:")
        print(res.text[:300])
        return []

    # 상품 리스트 추출
    try:
        goods_lists = data['pageProps']['dehydratedState']['queries'][1]['state']['data']['products']
    except KeyError:
        print("데이터 구조가 예상과 다름. keys:", data.keys())
        return []

    # 결과 저장
    result = []
    for goods in goods_lists:
        result.append({
            '상품ID': goods['id'],
            '브랜드명': goods['brandName'],
            '제품명': goods['name'],
        })

    return result



In [25]:
a = product_list()
a

응답 상태 코드: 200


[{'상품ID': 767440,
  '브랜드명': '헬로우슬립',
  '제품명': '[7일간,최대20%쿠폰]NEW컬러! 카스테라 워싱 옥수수솜 간절기/사계절 차렵이불세트'},
 {'상품ID': 212240,
  '브랜드명': '아엠홈',
  '제품명': '[5%쿠폰][맞춤]비침없는 도톰 레이스/쉬폰커튼(나비주름/핀형/봉집)'},
 {'상품ID': 264565, '브랜드명': '데코지오', '제품명': '맞춤 브라우니 린넨 암막커튼 핀형/아일렛형/형상기억 33색상'},
 {'상품ID': 169814,
  '브랜드명': '스타일링홈',
  '제품명': '[1+1]100% 완벽 자외선 빛차단 냉방유지 창문/거실/안방 아일렛/핀형 암막커튼'},
 {'상품ID': 676405,
  '브랜드명': '베이직톤',
  '제품명': '[단하루!20%쿠폰]호텔침구 클린코튼 고밀도60수 사계절/간절기 옥수수솜 차렵이불세트'},
 {'상품ID': 2636441,
  '브랜드명': '쁘리엘르',
  '제품명': '[단하루 역대급 혜택] NEW 재구매율 높은 퐁신퐁신 유어메이트 항균 차렵이불 풀세트'},
 {'상품ID': 33583, '브랜드명': '창안애', '제품명': '25mm 알루미늄 블라인드 거실 창문 35colors'},
 {'상품ID': 3316539, '브랜드명': '드파인', '제품명': '하이랜드 루프 사이잘룩 러그 7size'},
 {'상품ID': 338170,
  '브랜드명': '까르데코',
  '제품명': '메리미 화이트 이중 레이스 암막커튼 2장세트 핀형/아일렛형4size'},
 {'상품ID': 2370041,
  '브랜드명': '마틸라',
  '제품명': '[집요한세일,쿠폰최대20%+무배]NEW빌리호텔 먼지없는 차렵이불세트 23color'},
 {'상품ID': 1215312,
  '브랜드명': '체고루루',
  '제품명': '[오늘의딜]디아망 방수러그 먼지없는 거실 대형 워셔블 카페트 고양이 러그 18color'},
 {'상품ID': 1418521,
  '브

In [ ]:

def product_review(pages=10):
    products = product_list()
    all_reviews = []

    for prod in products:
        product_id = prod['상품ID']
        pro_result = []

        # 여러 페이지 리뷰 수집
        for i in range(3, pages + 1):  # 1페이지부터
            url = f"https://ohou.se/production_reviews.json?production_id={product_id}&page={i}&order=recent&photo_review_only="
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
                'Sec-Fetch-Dest': 'empty',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Site': 'same-origin'
            }
            res = requests.get(url, headers=headers)
            pro_result.append(res.json())

        # 리뷰 데이터 추출
        reviews = []
        for review_page in pro_result:
            for review_list in review_page['reviews']:  # 딕셔너리 접근 그대로
                reviews.append({
                    '리뷰ID': review_list['id'],
                    '상품ID': review_list['production_information']['id'],
                    '고객ID': review_list['writer_id'],
                    '고객닉네임': review_list['writer_nickname'],
                    '상품옵션': review_list['production_information']['explain'],
                    '별점': review_list['review']['star_avg'],
                    '작성내용': review_list['review']['comment'],
                    '작성날짜': review_list['created_at'],
                })

        all_reviews.extend(reviews)

    return all_reviews


In [28]:
my_reviews = product_review(pages=5)
my_reviews

응답 상태 코드: 200


[{'리뷰ID': 53303785,
  '상품ID': 767440,
  '고객ID': 13742260,
  '고객닉네임': '132456123',
  '상품옵션': '색상: 사계절용_스트라이프 딥그린 / 구성 및 사이즈: 슈퍼싱글 풀세트(SS)',
  '별점': 5.0,
  '작성내용': '시원하고 촉감도 좋고 좋아욥 ㅎㅎ 여러명 정보 물어봐서 알려줫더니 다들 구매하더라구욥',
  '작성날짜': '2025.08.19'},
 {'리뷰ID': 53303064,
  '상품ID': 767440,
  '고객ID': 812867,
  '고객닉네임': '공오일사',
  '상품옵션': '색상: 사계절용_코코넛크림 / 구성 및 사이즈: 퀸&킹겸용 풀세트(Q패드)',
  '별점': 4.0,
  '작성내용': '마음에 들어요',
  '작성날짜': '2025.08.19'},
 {'리뷰ID': 53302446,
  '상품ID': 767440,
  '고객ID': 6453159,
  '고객닉네임': '김은경💕2',
  '상품옵션': '색상: 사계절용_바닐라 / 구성 및 사이즈: 슈퍼싱글 이불베개세트(SS)',
  '별점': 5.0,
  '작성내용': '최고예요',
  '작성날짜': '2025.08.19'},
 {'리뷰ID': 53302445,
  '상품ID': 767440,
  '고객ID': 6453159,
  '고객닉네임': '김은경💕2',
  '상품옵션': '색상: 사계절용_올리브그린 / 구성 및 사이즈: 슈퍼싱글 이불베개세트(SS)',
  '별점': 5.0,
  '작성내용': '최고예요',
  '작성날짜': '2025.08.19'},
 {'리뷰ID': 53302234,
  '상품ID': 767440,
  '고객ID': 6265363,
  '고객닉네임': 'ㄱ.김예지',
  '상품옵션': '색상: 크림아이보리 / 구성 및 사이즈: 슈퍼싱글 이불베개세트(SS이불+베개커버1P)',
  '별점': 5.0,
  '작성내용': '택배 잘 받았구 아주 안전하게 와서 잘 받았습니다 빠른배송 안전

In [43]:
result[:2]

[{'상품ID': 767440,
  '브랜드명': '헬로우슬립',
  '제품명': '[7일간,최대20%쿠폰]NEW컬러! 카스테라 워싱 옥수수솜 간절기/사계절 차렵이불세트'},
 {'상품ID': 212240,
  '브랜드명': '아엠홈',
  '제품명': '[5%쿠폰][맞춤]비침없는 도톰 레이스/쉬폰커튼(나비주름/핀형/봉집)'}]